1. Read in the Dask Dataframe package

In [1]:
import dask.dataframe as dd

/usr/local/lib/python3.11/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


2. Lets read data from a public S3 bucket

In [4]:
#-- need this for reading from S3
!pip install s3fs

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.5/184.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.1/78.1 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 32.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2025.2.0 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda

In [5]:
import s3fs

In [6]:
ddf = dd.read_parquet(
    "s3://dask-data/nyc-taxi/nyc-2015.parquet/part.*.parquet",
    columns=["passenger_count", "tip_amount", "VendorID", "trip_distance"],
    storage_options={"anon": True},
)

3. Look at the first few rows

In [8]:
ddf.head()

,passenger_count,tip_amount,VendorID,trip_distance
0,5,0.0,1,4.00
1,3,0.0,2,1.56
2,1,0.0,2,1.68
3,1,0.0,1,0.80
4,2,0.0,2,2.57


4. How many records does the data have?

In [7]:
len(ddf)

146112989

5. Create a persisted ddf, where `tip_distance` is greater than 10

In [9]:
long_distance = ddf.loc[ddf["trip_distance"] > 10].persist()

6. Calculate the mean, max, and min of `tip_amount` seperately and compute the result

In [13]:
long_distance["tip_amount"].mean().compute()

6.059765222904728

In [14]:
long_distance["tip_amount"].min().compute()

0.0

In [15]:
long_distance["tip_amount"].max().compute()

910.05